# 1 import necessary libraries

In [180]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.metrics import accuracy_score,classification_report,precision_score,recall_score,f1_score,precision_recall_fscore_support,confusion_matrix
import warnings
import os 
import time 

In [181]:
#  model deployment used this code line 
print('program is running ....')
print()
start_time=time.time()

program is running ....



In [182]:
# load the datasets 
a1=pd.read_csv('case_study1.xlsx - case_study1.csv')
a2=pd.read_csv('case_study2.xlsx - case_study2.csv')

In [183]:
a1.shape,a2.shape

((51336, 26), (51336, 62))

In [184]:
df1=a1.copy()
df2=a2.copy()

In [185]:
df1

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
0,1,5,4,1,0,0,0.000,0.00,0.200,0.800,...,0,0,1,0,4,1,4,0,72,18
1,2,1,0,1,0,0,0.000,0.00,1.000,0.000,...,0,1,0,0,0,0,1,0,7,7
2,3,8,0,8,1,0,0.125,0.00,1.000,0.000,...,0,6,1,0,0,2,6,0,47,2
3,4,1,0,1,1,0,1.000,0.00,1.000,0.000,...,0,0,0,0,0,0,1,1,5,5
4,5,3,2,1,0,0,0.000,0.00,0.333,0.667,...,0,0,0,0,0,3,0,2,131,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51331,51332,3,0,3,1,0,0.333,0.00,1.000,0.000,...,0,2,0,0,0,0,3,1,24,5
51332,51333,4,2,2,0,1,0.000,0.25,0.500,0.500,...,0,2,0,0,0,2,2,0,74,7
51333,51334,2,1,1,1,1,0.500,0.50,0.500,0.500,...,0,2,0,0,0,0,2,0,9,5
51334,51335,2,1,1,0,0,0.000,0.00,0.500,0.500,...,0,2,0,0,0,0,2,0,15,8


In [186]:
df2

,PROSPECTID,time_since_recent_payment,time_since_first_deliquency,time_since_recent_deliquency,num_times_delinquent,max_delinquency_level,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,...,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,549,35,15,11,29,29,0,0,0,...,0.0,0.0,0.0,13.333,1,0,PL,PL,696,P2
1,2,47,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,0.860,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,302,11,3,9,25,25,1,9,8,...,0.0,0.0,0.0,5741.667,1,0,ConsumerLoan,others,693,P2
3,4,-99999,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,9.900,0,0,others,others,673,P2
4,5,583,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,-99999.000,0,0,AL,AL,753,P1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51331,51332,15,24,23,2,24,24,0,0,0,...,0.0,0.0,0.0,1.661,0,0,ConsumerLoan,ConsumerLoan,650,P4
51332,51333,57,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,0.520,0,0,others,others,702,P1
51333,51334,32,-99999,-99999,0,-99999,0,0,0,0,...,0.0,1.0,0.0,0.567,0,0,ConsumerLoan,others,661,P3
51334,51335,58,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,1.202,0,0,ConsumerLoan,others,686,P2


In [187]:
# remove the nulls value 
df1= df1.loc[df1['Age_Oldest_TL']!=-99999]

In [188]:
# remove the columns or row from the df2 data sets if data greater then 10000 rows is null them remove this respective columns 
columns_to_be_removed=[]

for i in df2.columns:
    if df2.loc[df2[i] == -99999].shape[0] > 10000:
        columns_to_be_removed.append(i)
        


In [189]:
df2=df2.drop(columns_to_be_removed,axis=1)
print(f"We have dropped {columns_to_be_removed}")

We have dropped ['time_since_first_deliquency', 'time_since_recent_deliquency', 'max_delinquency_level', 'max_deliq_6mts', 'max_deliq_12mts', 'CC_utilization', 'PL_utilization', 'max_unsec_exposure_inPct']


In [190]:
# less than 10000 rows is null to removes this rows 
for i in df2.columns:
    df2=df2.loc[df2[i] != -99999]

In [191]:
df2.isnull().sum()

PROSPECTID                    0
time_since_recent_payment     0
num_times_delinquent          0
max_recent_level_of_deliq     0
num_deliq_6mts                0
num_deliq_12mts               0
num_deliq_6_12mts             0
num_times_30p_dpd             0
num_times_60p_dpd             0
num_std                       0
num_std_6mts                  0
num_std_12mts                 0
num_sub                       0
num_sub_6mts                  0
num_sub_12mts                 0
num_dbt                       0
num_dbt_6mts                  0
num_dbt_12mts                 0
num_lss                       0
num_lss_6mts                  0
num_lss_12mts                 0
recent_level_of_deliq         0
tot_enq                       0
CC_enq                        0
CC_enq_L6m                    0
CC_enq_L12m                   0
PL_enq                        0
PL_enq_L6m                    0
PL_enq_L12m                   0
time_since_recent_enq         0
enq_L12m                      0
enq_L6m 

# 

## merge the two data sets in common columns  df1 and df2 

In [192]:
# checking the common column name in df1 and df2 
for i in list(df1.columns):
    for i in list(df2.columns):
        print(i)

PROSPECTID
time_since_recent_payment
num_times_delinquent
max_recent_level_of_deliq
num_deliq_6mts
num_deliq_12mts
num_deliq_6_12mts
num_times_30p_dpd
num_times_60p_dpd
num_std
num_std_6mts
num_std_12mts
num_sub
num_sub_6mts
num_sub_12mts
num_dbt
num_dbt_6mts
num_dbt_12mts
num_lss
num_lss_6mts
num_lss_12mts
recent_level_of_deliq
tot_enq
CC_enq
CC_enq_L6m
CC_enq_L12m
PL_enq
PL_enq_L6m
PL_enq_L12m
time_since_recent_enq
enq_L12m
enq_L6m
enq_L3m
MARITALSTATUS
EDUCATION
AGE
GENDER
NETMONTHLYINCOME
Time_With_Curr_Empr
pct_of_active_TLs_ever
pct_opened_TLs_L6m_of_L12m
pct_currentBal_all_TL
CC_Flag
PL_Flag
pct_PL_enq_L6m_of_L12m
pct_CC_enq_L6m_of_L12m
pct_PL_enq_L6m_of_ever
pct_CC_enq_L6m_of_ever
HL_Flag
GL_Flag
last_prod_enq2
first_prod_enq2
Credit_Score
Approved_Flag
PROSPECTID
time_since_recent_payment
num_times_delinquent
max_recent_level_of_deliq
num_deliq_6mts
num_deliq_12mts
num_deliq_6_12mts
num_times_30p_dpd
num_times_60p_dpd
num_std
num_std_6mts
num_std_12mts
num_sub
num_sub_6mts
num

In [193]:
# merge the two datframe inner join so that no nulls are present 
df=pd.merge(df1,df2,how='inner',left_on=['PROSPECTID'],right_on=['PROSPECTID'])

In [194]:
df.shape

(42064, 79)

In [195]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 79 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PROSPECTID                  42064 non-null  int64  
 1   Total_TL                    42064 non-null  int64  
 2   Tot_Closed_TL               42064 non-null  int64  
 3   Tot_Active_TL               42064 non-null  int64  
 4   Total_TL_opened_L6M         42064 non-null  int64  
 5   Tot_TL_closed_L6M           42064 non-null  int64  
 6   pct_tl_open_L6M             42064 non-null  float64
 7   pct_tl_closed_L6M           42064 non-null  float64
 8   pct_active_tl               42064 non-null  float64
 9   pct_closed_tl               42064 non-null  float64
 10  Total_TL_opened_L12M        42064 non-null  int64  
 11  Tot_TL_closed_L12M          42064 non-null  int64  
 12  pct_tl_open_L12M            42064 non-null  float64
 13  pct_tl_closed_L12M          420

In [196]:
df.isna().sum().sum()

0

In [197]:
df.head()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.000,0.0,0.200,0.800,...,0.0,0.0,0.000,0.0,1,0,PL,PL,696,P2
1,2,1,0,1,0,0,0.000,0.0,1.000,0.000,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,8,0,8,1,0,0.125,0.0,1.000,0.000,...,0.0,0.0,0.000,0.0,1,0,ConsumerLoan,others,693,P2
3,5,3,2,1,0,0,0.000,0.0,0.333,0.667,...,0.0,0.0,0.000,0.0,0,0,AL,AL,753,P1
4,6,6,5,1,0,0,0.000,0.0,0.167,0.833,...,1.0,0.0,0.429,0.0,1,0,ConsumerLoan,PL,668,P3


In [198]:
# check the how many columns are categorical
for i in df.columns:
    if df[i].dtype=='object':
        print(i)

MARITALSTATUS
EDUCATION
GENDER
last_prod_enq2
first_prod_enq2
Approved_Flag


In [199]:
# check allvalues one by one how many value this respective columns 
print(df['MARITALSTATUS'].value_counts())
print(df['EDUCATION'].value_counts())
print(df['GENDER'].value_counts())
print(df['last_prod_enq2'].value_counts())
print(df['first_prod_enq2'].value_counts())
print(df['Approved_Flag'].value_counts())

MARITALSTATUS
Married    30886
Single     11178
Name: count, dtype: int64
EDUCATION
GRADUATE          14140
12TH              11703
SSC                7241
UNDER GRADUATE     4572
OTHERS             2291
POST-GRADUATE      1898
PROFESSIONAL        219
Name: count, dtype: int64
GENDER
M    37345
F     4719
Name: count, dtype: int64
last_prod_enq2
ConsumerLoan    16480
others          13653
PL               7553
CC               2195
AL               1353
HL                830
Name: count, dtype: int64
first_prod_enq2
others          20640
ConsumerLoan    11075
PL               4431
AL               2641
CC               1988
HL               1289
Name: count, dtype: int64
Approved_Flag
P2    25452
P3     6440
P4     5264
P1     4908
Name: count, dtype: int64


In [200]:
# chi square testing  for frature selection for categorical variables 
for i in ['MARITALSTATUS','EDUCATION','GENDER','last_prod_enq2','first_prod_enq2']:
    chi2,pval,_,_=chi2_contingency(pd.crosstab(df[i],df['Approved_Flag']))
    print(i,'----',pval)

MARITALSTATUS ---- 3.578180861038862e-233
EDUCATION ---- 2.6942265249737532e-30
GENDER ---- 1.907936100186563e-05
last_prod_enq2 ---- 0.0
first_prod_enq2 ---- 7.84997610555419e-287


### since all the categorical features have pval<=0.05 we will accept all

In [201]:
# vif test of numerical columns 
# now move on test of numerical features this variable is predictabele to other variabale to kickout and not predictable put numeric_columns
numeric_columns=[]
for i in df.columns:
    if df[i].dtype != 'object' and i not in ['PROSPECTID','Approved_Flag']:
        numeric_columns.append(i)

In [202]:
# VIF sequentially check
# calculate the vif value all numerical columns and vif value vif<=6 accept this columns oherwise drop this columns 
vif_data=df[numeric_columns]
total_columns=vif_data.shape[1]
columns_to_be_kept=[]
column_index= 0

for i in range(0,total_columns):
    vif_value=variance_inflation_factor(vif_data,column_index)
    print(column_index,'----',vif_value)
    
    if vif_value <= 6:
        columns_to_be_kept.append(numeric_columns[i])
        column_index =column_index+1
    else:
        vif_data=vif_data.drop([numeric_columns[i]],axis=1)

C:\Users\ranig\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 ---- inf


C:\Users\ranig\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 ---- inf
0 ---- 11.320180023967996
0 ---- 8.363698035000327
0 ---- 6.520647877790928
0 ---- 5.149501618212625
1 ---- 2.611111040579735


C:\Users\ranig\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


2 ---- inf
2 ---- 1788.7926256209232
2 ---- 8.601028256477228
2 ---- 3.8328007921530785
3 ---- 6.099653381646739
3 ---- 5.5813520096427585
4 ---- 1.985584353098778


C:\Users\ranig\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


5 ---- inf
5 ---- 4.809538302819343
6 ---- 23.270628983464636
6 ---- 30.595522588100053
6 ---- 4.3843464059655854
7 ---- 3.0646584155234238
8 ---- 2.898639771299252
9 ---- 4.377876915347324
10 ---- 2.2078535836958433
11 ---- 4.916914200506864
12 ---- 5.214702030064725
13 ---- 3.3861625024231476
14 ---- 7.840583309478997
14 ---- 5.255034641721438


C:\Users\ranig\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:195: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


15 ---- inf
15 ---- 7.380634506427232
15 ---- 1.421005001517573
16 ---- 8.083255010190323
16 ---- 1.624122752404011
17 ---- 7.257811920140003
17 ---- 15.59624383268298
17 ---- 1.825857047132431
18 ---- 1.5080839450032664
19 ---- 2.172088834824577
20 ---- 2.623397553527229
21 ---- 2.2959970812106176
22 ---- 7.360578319196439
22 ---- 2.1602387773102554
23 ---- 2.8686288267891467
24 ---- 6.458218003637272
24 ---- 2.8474118865638265
25 ---- 4.7531981562840855
26 ---- 16.22735475594825
26 ---- 6.424377256363877
26 ---- 8.887080381808687
26 ---- 2.3804746142952653
27 ---- 8.60951347651454
27 ---- 13.06755093547673
27 ---- 3.5000400566546555
28 ---- 1.9087955874813773
29 ---- 17.006562234161628
29 ---- 10.730485153719197
29 ---- 2.3538497522950275
30 ---- 22.104855915136433
30 ---- 2.7971639638512906
31 ---- 3.424171203217696
32 ---- 10.175021454450935
32 ---- 6.408710354561301
32 ---- 1.0011511962625619
33 ---- 3.069197305397274
34 ---- 2.8091261600643724
35 ---- 20.249538381980678
35 ---- 1

##### result numeric all 72 cols are 32 is preditable but not unique among those 39

In [203]:
# check the anova for columns to be kept
from scipy.stats import f_oneway

columns_to_be_kept_numerical = []

for i in columns_to_be_kept:
    a = list(df[i])  
    b = list(df['Approved_Flag'])  
    
    group_P1 = [value for value, group in zip(a, b) if group == 'P1']
    group_P2 = [value for value, group in zip(a, b) if group == 'P2']
    group_P3 = [value for value, group in zip(a, b) if group == 'P3']
    group_P4 = [value for value, group in zip(a, b) if group == 'P4']


    f_statistic, p_value = f_oneway(group_P1, group_P2, group_P3, group_P4)

    if p_value <= 0.05:
        columns_to_be_kept_numerical.append(i)

In [204]:
# listing all the final features
features = columns_to_be_kept_numerical + ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']
df = df[features + ['Approved_Flag']]


In [205]:
df.columns

Index(['pct_tl_open_L6M', 'pct_tl_closed_L6M', 'Tot_TL_closed_L12M',
       'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'CC_TL', 'Home_TL', 'PL_TL',
       'Secured_TL', 'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL',
       'Age_Newest_TL', 'time_since_recent_payment',
       'max_recent_level_of_deliq', 'num_deliq_6_12mts', 'num_times_60p_dpd',
       'num_std_12mts', 'num_sub', 'num_sub_6mts', 'num_sub_12mts', 'num_dbt',
       'num_dbt_12mts', 'num_lss', 'recent_level_of_deliq', 'CC_enq_L12m',
       'PL_enq_L12m', 'time_since_recent_enq', 'enq_L3m', 'NETMONTHLYINCOME',
       'Time_With_Curr_Empr', 'CC_Flag', 'PL_Flag', 'pct_PL_enq_L6m_of_ever',
       'pct_CC_enq_L6m_of_ever', 'HL_Flag', 'GL_Flag', 'MARITALSTATUS',
       'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2',
       'Approved_Flag'],
      dtype='object')

In [206]:

# Label encoding for the categorical features
['MARITALSTATUS', 'EDUCATION', 'GENDER' , 'last_prod_enq2' ,'first_prod_enq2']

print(df['MARITALSTATUS'].unique())
print(df['EDUCATION'].unique())
print(df['GENDER'].unique())
print(df['last_prod_enq2'].unique())
print(df['first_prod_enq2'].unique())

['Married' 'Single']
['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
 'PROFESSIONAL']
['M' 'F']
['PL' 'ConsumerLoan' 'AL' 'CC' 'others' 'HL']
['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']


In [207]:
# Ordinal feature -- EDUCATION columns 
# SSC            : 1
# 12TH           : 2
# GRADUATE       : 3
# UNDER GRADUATE : 3
# POST-GRADUATE  : 4
# OTHERS         : 1
# PROFESSIONAL   : 3

# Others has to be verified by the business end user

df.loc[df['EDUCATION'] == 'SSC',['EDUCATION']]              = 1
df.loc[df['EDUCATION'] == '12TH',['EDUCATION']]             = 2
df.loc[df['EDUCATION'] == 'GRADUATE',['EDUCATION']]         = 3
df.loc[df['EDUCATION'] == 'UNDER GRADUATE',['EDUCATION']]   = 3
df.loc[df['EDUCATION'] == 'POST-GRADUATE',['EDUCATION']]    = 4
df.loc[df['EDUCATION'] == 'OTHERS',['EDUCATION']]           = 1
df.loc[df['EDUCATION'] == 'PROFESSIONAL',['EDUCATION']]     = 3


In [208]:
print(df['EDUCATION'].value_counts())
df['EDUCATION'] = df['EDUCATION'].astype(int)
df.info()

EDUCATION
3    18931
2    11703
1     9532
4     1898
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   pct_tl_open_L6M            42064 non-null  float64
 1   pct_tl_closed_L6M          42064 non-null  float64
 2   Tot_TL_closed_L12M         42064 non-null  int64  
 3   pct_tl_closed_L12M         42064 non-null  float64
 4   Tot_Missed_Pmnt            42064 non-null  int64  
 5   CC_TL                      42064 non-null  int64  
 6   Home_TL                    42064 non-null  int64  
 7   PL_TL                      42064 non-null  int64  
 8   Secured_TL                 42064 non-null  int64  
 9   Unsecured_TL               42064 non-null  int64  
 10  Other_TL                   42064 non-null  int64  
 11  Age_Oldest_TL              42064 non-null  int64  
 12  Age_Newest_TL         

In [209]:
# one hot encoding 
df_encoded=pd.get_dummies(df,columns=['MARITALSTATUS','GENDER','last_prod_enq2','first_prod_enq2'])
df_encoded.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 55 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pct_tl_open_L6M               42064 non-null  float64
 1   pct_tl_closed_L6M             42064 non-null  float64
 2   Tot_TL_closed_L12M            42064 non-null  int64  
 3   pct_tl_closed_L12M            42064 non-null  float64
 4   Tot_Missed_Pmnt               42064 non-null  int64  
 5   CC_TL                         42064 non-null  int64  
 6   Home_TL                       42064 non-null  int64  
 7   PL_TL                         42064 non-null  int64  
 8   Secured_TL                    42064 non-null  int64  
 9   Unsecured_TL                  42064 non-null  int64  
 10  Other_TL                      42064 non-null  int64  
 11  Age_Oldest_TL                 42064 non-null  int64  
 12  Age_Newest_TL                 42064 non-null  int64  
 13  t

In [210]:
df_encoded.shape

(42064, 55)

In [211]:
df_encoded.describe()

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,...,enq_L3m,NETMONTHLYINCOME,Time_With_Curr_Empr,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,EDUCATION
count,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,...,42064.000000,4.206400e+04,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000
mean,0.179032,0.097783,0.825504,0.160365,0.525746,0.145921,0.076241,0.328000,2.921334,2.341646,...,1.230458,2.692990e+04,110.345783,0.102962,0.193063,0.195497,0.064186,0.252235,0.056580,2.313689
std,0.278043,0.210957,1.537208,0.258831,1.106442,0.549314,0.358582,0.916368,6.379764,3.405397,...,2.069461,2.084300e+04,75.629967,0.303913,0.394707,0.367414,0.225989,0.434300,0.231042,0.871070
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.800000e+04,61.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,...,1.000000,2.400000e+04,92.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,0.333000,0.100000,1.000000,0.250000,1.000000,0.000000,0.000000,0.000000,3.000000,3.000000,...,2.000000,3.100000e+04,131.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,3.000000
max,1.000000,1.000000,33.000000,1.000000,34.000000,27.000000,10.000000,29.000000,235.000000,55.000000,...,42.000000,2.500000e+06,1020.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000


# machine learning model fitting 

### 1 random forest 

In [212]:
y=df_encoded['Approved_Flag']
x=df_encoded.drop(['Approved_Flag'],axis=1)

In [213]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [214]:
rf_classifiers=RandomForestClassifier(n_estimators=200,random_state=42)
rf_classifiers.fit(x_train,y_train)

RandomForestClassifier(n_estimators=200, random_state=42)

In [215]:
y_pred=rf_classifiers.predict(x_test)

In [216]:
accuracy=accuracy_score(y_test,y_pred)
print(f'accuracy:{accuracy}')

accuracy:0.7636990372043266


In [217]:
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

In [218]:
for i ,v in enumerate(['p1','p2','p3','p4']):
    print(f'class {v}:')
    print(f'precision: {precision[i]}:')
    print(f'recall:    {recall[i]}:')
    print(f'f1_score:  {f1_score[i]}:')

class p1:
precision: 0.8370457209847597:
recall:    0.7041420118343196:
f1_score:  0.7648634172469202:
class p2:
precision: 0.7957519116397621:
recall:    0.9282457879088206:
f1_score:  0.856907593778591:
class p3:
precision: 0.4423380726698262:
recall:    0.21132075471698114:
f1_score:  0.28600612870275793:
class p4:
precision: 0.7178502879078695:
recall:    0.7269193391642371:
f1_score:  0.7223563495895703:


### 2 Xg Boost 

In [219]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

xgb_classifier = xgb.XGBClassifier(objective='multi:softmax',  num_class=4)



y = df_encoded['Approved_Flag']
x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

xgb_classifier.fit(x_train, y_train)
y_pred=xgb_classifier.predict(x_test)
accuracy=accuracy_score(y_test,y_pred)
print(f'accuracy_score: {accuracy:.2f}')

accuracy_score: 0.78


In [220]:
precision,recall,f1score,_=precision_recall_fscore_support(y_test,y_pred)

In [221]:
for i,v in enumerate(['p1','p2','p3','p4']):
    print(f'class:{v}:')
    print(f'precision:{precision[i]}:')
    print(f'recall:{recall[i]}:')
    print(f'f1score:{f1score[i]}:')

class:p1:
precision:0.823906083244397:
recall:0.7613412228796844:
f1score:0.7913890312660175:
class:p2:
precision:0.8255418233924413:
recall:0.913577799801784:
f1score:0.8673315769665035:
class:p3:
precision:0.4756380510440835:
recall:0.30943396226415093:
f1score:0.37494284407864653:
class:p4:
precision:0.7342386032977691:
recall:0.7356656948493683:
f1score:0.7349514563106796:


### 3 .Decision tree

In [222]:
from sklearn.tree import DecisionTreeClassifier
dt_model=DecisionTreeClassifier(max_depth=20,min_samples_split=10)

In [223]:
x=df_encoded.drop(['Approved_Flag'],axis=1)
y=df_encoded['Approved_Flag']


In [224]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
dt_model.fit(x_train,y_train)
y_pred=dt_model.predict(x_test)
accuracy=accuracy_score(y_test,y_pred)
print(f"Accuracy: {accuracy:.2f}")
precision,recall,f1score,_=precision_recall_fscore_support(y_test,y_pred)

Accuracy: 0.71


In [225]:
for i,v in enumerate(['p1','p2','p3','p4']):
    print(f'class:{v}:')
    print(f'precision:{precision[i]}:')
    print(f'recall:{recall[i]}:')
    print(f'f1score:{f1score[i]}:')

class:p1:
precision:0.7217898832684825:
recall:0.7317554240631163:
f1score:0.7267384916748286:
class:p2:
precision:0.8094773790951638:
recall:0.822794846382557:
f1score:0.8160817851174678:
class:p3:
precision:0.3404592240696754:
recall:0.32452830188679244:
f1score:0.3323029366306028:
class:p4:
precision:0.6549295774647887:
recall:0.6326530612244898:
f1score:0.643598615916955:


### xgboost is giving me best results 
### we will further fintune it 

In [226]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 55 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pct_tl_open_L6M               42064 non-null  float64
 1   pct_tl_closed_L6M             42064 non-null  float64
 2   Tot_TL_closed_L12M            42064 non-null  int64  
 3   pct_tl_closed_L12M            42064 non-null  float64
 4   Tot_Missed_Pmnt               42064 non-null  int64  
 5   CC_TL                         42064 non-null  int64  
 6   Home_TL                       42064 non-null  int64  
 7   PL_TL                         42064 non-null  int64  
 8   Secured_TL                    42064 non-null  int64  
 9   Unsecured_TL                  42064 non-null  int64  
 10  Other_TL                      42064 non-null  int64  
 11  Age_Oldest_TL                 42064 non-null  int64  
 12  Age_Newest_TL                 42064 non-null  int64  
 13  t

In [230]:
##Applying standard scaler

from sklearn.preprocessing import StandardScaler

columns_to_be_scaled = ['Age_Newest_TL','time_since_recent_payment',
'max_recent_level_of_deliq','recent_level_of_deliq',
'time_since_recent_enq','NETMONTHLYINCOME','Time_With_Curr_Empr']

for i in columns_to_be_scaled:
    column_data = df_encoded[i].values.reshape(-1, 1)
    scaler = StandardScaler()
    scaled_column = scaler.fit_transform(column_data)
    df_encoded[i] = scaled_column

In [231]:
xgb_classifier = xgb.XGBClassifier(objective='multi:softmax',  num_class=4)

In [232]:
y = df_encoded['Approved_Flag']
x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

In [233]:
xgb_classifier.fit(x_train, y_train)
y_pred = xgb_classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.78


In [234]:
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()

Class p1:
Precision: 0.823906083244397
Recall: 0.7613412228796844
F1 Score: 0.7913890312660175

Class p2:
Precision: 0.8255418233924413
Recall: 0.913577799801784
F1 Score: 0.8673315769665035

Class p3:
Precision: 0.4756380510440835
Recall: 0.30943396226415093
F1 Score: 0.37494284407864653

Class p4:
Precision: 0.7342386032977691
Recall: 0.7356656948493683
F1 Score: 0.7349514563106796



## Hyperparameter tuning in XGboost

In [235]:
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

In [236]:
## Define the XGBClassifier with the initial set of hyperparameters
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=4)

In [237]:
# Define the parameter grid for hyperparameter tuning

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

C:\Users\ranig\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
57 fits failed out of a total of 81.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ranig\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ranig\anaconda3\lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
  File "C:\Users\ranig\anaconda3\lib\site-packages\xgboost\sklearn.py", line 1466, in fit
    with config_context(verbosity=self.verbosity):
  File "C:\Users\ranig\anaconda3\lib\contextlib.py", l

Best Hyperparameters: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 50}


In [238]:
# Evaluate the model with the best hyperparameters on the test set
best_model = grid_search.best_estimator_
accuracy = best_model.score(x_test, y_test)
print("Test Accuracy:", accuracy)


Test Accuracy: 0.7480090336384168


## hyperparameter tuning using for xgboost  (defining the Grid)


In [239]:
param_grid={
    'colsample_bytree':[0.1,0.3,0.5,0.7,0.9],
    'learning_rate'   :[0.001,0.01,0.1,1],
    'max_depth'       :[3,5,8,10],
    'alpha'           :[1,10,100],
    'n_estimators'    :[10,50,100]
}

In [241]:
index=0
answers_grid={
    'combination'     :[],
    'train_accuracy'  :[],
    'test_accuracy'   :[],
    'colsample_bytree':[],
    'learning_rate'   :[],
    'max_depth'       :[],
    'alpha'           :[],
    'n_estimators'    :[]
}

In [242]:
# loop through each combination of hyperparameter
for colsample_bytree in param_grid['colsample_bytree']:
    for learning_rate in param_grid['learning_rate']:
        for max_depth in param_grid['max_depth']:
            for alpha in param_grid['alpha']:
                for n_estimators in param_grid['n_estimators']:
                    
                    index=index + 1
                    
                    # Correctly define and encode the target variable
                    y = df_encoded['Approved_Flag']
                    x = df_encoded.drop(['Approved_Flag'], axis=1)

                    # Apply LabelEncoder on the target variable 'y'
                    label_encoder = LabelEncoder()
                    y_encoded = label_encoder.fit_transform(y)  # Apply LabelEncoder on 'y'

                    # Train-test split
                    x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

                    # Train the XGBoost model
                    model = xgb.XGBRFClassifier(objective='multi:softmax', num_class=4, colsample_bytree=colsample_bytree, 
                                                learning_rate=learning_rate, max_depth=max_depth, alpha=alpha, 
                                                n_estimators=n_estimators)

                    model.fit(x_train, y_train)
                   
                    # predict training and testing sets
                    y_prediction_train=model.predict(x_train)
                    y_prediction_test=model.predict(x_test)
                    
                    # calculate the train and testing result
                    train_accuracy = accuracy_score(y_train,y_prediction_train)
                    test_accuracy = accuracy_score(y_test,y_prediction_test)

                    # included into the list 
                    
                    answers_grid['combination'].append(index)
                    answers_grid['train_accuracy'].append(train_accuracy)
                    answers_grid['test_accuracy'].append(test_accuracy)
                    answers_grid['colsample_bytree'].append(colsample_bytree)
                    answers_grid['learning_rate'].append(learning_rate)
                    answers_grid['max_depth'].append(max_depth)
                    answers_grid['alpha'].append(alpha)
                    answers_grid['n_estimators'].append(n_estimators)
                    
                    # print and results
                    print(f'combination{index}')
                    print(f'colsample_bytree:{colsample_bytree}, learning_rate:{learning_rate},max_depth:{max_depth},alpha:{alpha},n_estimators:{n_estimators}')
                    print(f'train_accuracy:{train_accuracy}')
                    print(f'test_accuracy:{test_accuracy}')
                    print("-"*30)

combination1
colsample_bytree:0.1, learning_rate:0.001,max_depth:3,alpha:1,n_estimators:10
train_accuracy:0.606519865680069
test_accuracy:0.5997860454059194
------------------------------
combination2
colsample_bytree:0.1, learning_rate:0.001,max_depth:3,alpha:1,n_estimators:50
train_accuracy:0.6064307152833497
test_accuracy:0.5996671817425413
------------------------------
combination3
colsample_bytree:0.1, learning_rate:0.001,max_depth:3,alpha:1,n_estimators:100
train_accuracy:0.6064307152833497
test_accuracy:0.5996671817425413
------------------------------
combination4
colsample_bytree:0.1, learning_rate:0.001,max_depth:3,alpha:10,n_estimators:10
train_accuracy:0.6064307152833497
test_accuracy:0.5996671817425413
------------------------------
combination5
colsample_bytree:0.1, learning_rate:0.001,max_depth:3,alpha:10,n_estimators:50
train_accuracy:0.6064307152833497
test_accuracy:0.5996671817425413
------------------------------
combination6
colsample_bytree:0.1, learning_rate:0.00

combination44
colsample_bytree:0.1, learning_rate:0.01,max_depth:3,alpha:100,n_estimators:50
train_accuracy:0.6064307152833497
test_accuracy:0.5996671817425413
------------------------------
combination45
colsample_bytree:0.1, learning_rate:0.01,max_depth:3,alpha:100,n_estimators:100
train_accuracy:0.6064307152833497
test_accuracy:0.5996671817425413
------------------------------
combination46
colsample_bytree:0.1, learning_rate:0.01,max_depth:5,alpha:1,n_estimators:10
train_accuracy:0.607084484859291
test_accuracy:0.6001426363960537
------------------------------
combination47
colsample_bytree:0.1, learning_rate:0.01,max_depth:5,alpha:1,n_estimators:50
train_accuracy:0.6066090160767882
test_accuracy:0.5996671817425413
------------------------------
combination48
colsample_bytree:0.1, learning_rate:0.01,max_depth:5,alpha:1,n_estimators:100
train_accuracy:0.6072925024516359
test_accuracy:0.5999049090692975
------------------------------
combination49
colsample_bytree:0.1, learning_rate:

combination88
colsample_bytree:0.1, learning_rate:0.1,max_depth:5,alpha:100,n_estimators:10
train_accuracy:0.6064604320822561
test_accuracy:0.5996671817425413
------------------------------
combination89
colsample_bytree:0.1, learning_rate:0.1,max_depth:5,alpha:100,n_estimators:50
train_accuracy:0.6064307152833497
test_accuracy:0.5996671817425413
------------------------------
combination90
colsample_bytree:0.1, learning_rate:0.1,max_depth:5,alpha:100,n_estimators:100
train_accuracy:0.6064307152833497
test_accuracy:0.5996671817425413
------------------------------
combination91
colsample_bytree:0.1, learning_rate:0.1,max_depth:8,alpha:1,n_estimators:10
train_accuracy:0.6080354224242964
test_accuracy:0.6003803637228099
------------------------------
combination92
colsample_bytree:0.1, learning_rate:0.1,max_depth:8,alpha:1,n_estimators:50
train_accuracy:0.6102047487444653
test_accuracy:0.6020444550101034
------------------------------
combination93
colsample_bytree:0.1, learning_rate:0.1

combination132
colsample_bytree:0.1, learning_rate:1,max_depth:8,alpha:10,n_estimators:100
train_accuracy:0.6064604320822561
test_accuracy:0.5996671817425413
------------------------------
combination133
colsample_bytree:0.1, learning_rate:1,max_depth:8,alpha:100,n_estimators:10
train_accuracy:0.6064307152833497
test_accuracy:0.5996671817425413
------------------------------
combination134
colsample_bytree:0.1, learning_rate:1,max_depth:8,alpha:100,n_estimators:50
train_accuracy:0.6064307152833497
test_accuracy:0.5996671817425413
------------------------------
combination135
colsample_bytree:0.1, learning_rate:1,max_depth:8,alpha:100,n_estimators:100
train_accuracy:0.6064307152833497
test_accuracy:0.5996671817425413
------------------------------
combination136
colsample_bytree:0.1, learning_rate:1,max_depth:10,alpha:1,n_estimators:10
train_accuracy:0.6174556476776322
test_accuracy:0.6089385474860335
------------------------------
combination137
colsample_bytree:0.1, learning_rate:1,ma

combination175
colsample_bytree:0.3, learning_rate:0.001,max_depth:10,alpha:10,n_estimators:10
train_accuracy:0.6755222727407804
test_accuracy:0.6638535599667181
------------------------------
combination176
colsample_bytree:0.3, learning_rate:0.001,max_depth:10,alpha:10,n_estimators:50
train_accuracy:0.6926094321119729
test_accuracy:0.6831094734339712
------------------------------
combination177
colsample_bytree:0.3, learning_rate:0.001,max_depth:10,alpha:10,n_estimators:100
train_accuracy:0.685358533178806
test_accuracy:0.6741946986806133
------------------------------
combination178
colsample_bytree:0.3, learning_rate:0.001,max_depth:10,alpha:100,n_estimators:10
train_accuracy:0.6504115776648539
test_accuracy:0.6450731011529776
------------------------------
combination179
colsample_bytree:0.3, learning_rate:0.001,max_depth:10,alpha:100,n_estimators:50
train_accuracy:0.653234673560964
test_accuracy:0.6475692380839178
------------------------------
combination180
colsample_bytree:0.

...

combination522
colsample_bytree:0.7, learning_rate:0.1,max_depth:5,alpha:100,n_estimators:100
train_accuracy:0.7237229205669965
test_accuracy:0.7218590276952336
------------------------------
combination523
colsample_bytree:0.7, learning_rate:0.1,max_depth:8,alpha:1,n_estimators:10
train_accuracy:0.7885649757808089
test_accuracy:0.7590633543325805
------------------------------
combination524
colsample_bytree:0.7, learning_rate:0.1,max_depth:8,alpha:1,n_estimators:50
train_accuracy:0.7910314700900419
test_accuracy:0.7634613098775704
------------------------------
combination525
colsample_bytree:0.7, learning_rate:0.1,max_depth:8,alpha:1,n_estimators:100
train_accuracy:0.7940923003774033
test_accuracy:0.7631047188874361
------------------------------
combination526
colsample_bytree:0.7, learning_rate:0.1,max_depth:8,alpha:10,n_estimators:10
train_accuracy:0.7769754242073044
test_accuracy:0.7603708546297396
------------------------------
combination527
colsample_bytree:0.7, learning_rate

combination566
colsample_bytree:0.7, learning_rate:1,max_depth:8,alpha:100,n_estimators:50
train_accuracy:0.7361742593087872
test_accuracy:0.7299417568049448
------------------------------
combination567
colsample_bytree:0.7, learning_rate:1,max_depth:8,alpha:100,n_estimators:100
train_accuracy:0.7356096401295652
test_accuracy:0.7269701652204921
------------------------------
combination568
colsample_bytree:0.7, learning_rate:1,max_depth:10,alpha:1,n_estimators:10
train_accuracy:0.8232741969035096
test_accuracy:0.7632235825508142
------------------------------
combination569
colsample_bytree:0.7, learning_rate:1,max_depth:10,alpha:1,n_estimators:50
train_accuracy:0.8269887967668123
test_accuracy:0.767502674432426
------------------------------
combination570
colsample_bytree:0.7, learning_rate:1,max_depth:10,alpha:1,n_estimators:100
train_accuracy:0.8274345487504086
test_accuracy:0.7698799476999881
------------------------------
combination571
colsample_bytree:0.7, learning_rate:1,max_

combination609
colsample_bytree:0.9, learning_rate:0.001,max_depth:10,alpha:10,n_estimators:100
train_accuracy:0.7921309916495795
test_accuracy:0.7682158564126946
------------------------------
combination610
colsample_bytree:0.9, learning_rate:0.001,max_depth:10,alpha:100,n_estimators:10
train_accuracy:0.7497251196101156
test_accuracy:0.7388565315583027
------------------------------
combination611
colsample_bytree:0.9, learning_rate:0.001,max_depth:10,alpha:100,n_estimators:50
train_accuracy:0.748833615642923
test_accuracy:0.7374301675977654
------------------------------
combination612
colsample_bytree:0.9, learning_rate:0.001,max_depth:10,alpha:100,n_estimators:100
train_accuracy:0.748625598050578
test_accuracy:0.7388565315583027
------------------------------
combination613
colsample_bytree:0.9, learning_rate:0.01,max_depth:3,alpha:1,n_estimators:10
train_accuracy:0.7206026566818222
test_accuracy:0.7168667538333532
------------------------------
combination614
colsample_bytree:0.9

combination652
colsample_bytree:0.9, learning_rate:0.1,max_depth:3,alpha:10,n_estimators:10
train_accuracy:0.7202460550949452
test_accuracy:0.7163912991798407
------------------------------
combination653
colsample_bytree:0.9, learning_rate:0.1,max_depth:3,alpha:10,n_estimators:50
train_accuracy:0.7189682327419691
test_accuracy:0.7147272078925473
------------------------------
combination654
colsample_bytree:0.9, learning_rate:0.1,max_depth:3,alpha:10,n_estimators:100
train_accuracy:0.7171852248075837
test_accuracy:0.7119933436348508
------------------------------
combination655
colsample_bytree:0.9, learning_rate:0.1,max_depth:3,alpha:100,n_estimators:10
train_accuracy:0.7082999019345636
test_accuracy:0.7053369784856769
------------------------------
combination656
colsample_bytree:0.9, learning_rate:0.1,max_depth:3,alpha:100,n_estimators:50
train_accuracy:0.7052390716472021
test_accuracy:0.7033162962082491
------------------------------
combination657
colsample_bytree:0.9, learning_r

combination696
colsample_bytree:0.9, learning_rate:1,max_depth:5,alpha:1,n_estimators:100
train_accuracy:0.7559062137826513
test_accuracy:0.7463449423511233
------------------------------
combination697
colsample_bytree:0.9, learning_rate:1,max_depth:5,alpha:10,n_estimators:10
train_accuracy:0.7496359692133964
test_accuracy:0.740520622845596
------------------------------
combination698
colsample_bytree:0.9, learning_rate:1,max_depth:5,alpha:10,n_estimators:50
train_accuracy:0.7524887819084128
test_accuracy:0.7443242600736955
------------------------------
combination699
colsample_bytree:0.9, learning_rate:1,max_depth:5,alpha:10,n_estimators:100
train_accuracy:0.7537368874624826
test_accuracy:0.7453940330440985
------------------------------
combination700
colsample_bytree:0.9, learning_rate:1,max_depth:5,alpha:100,n_estimators:10
train_accuracy:0.7347775697601854
test_accuracy:0.7281588018542732
------------------------------
combination701
colsample_bytree:0.9, learning_rate:1,max_de

# to check the model accuracy of unseen data sets 

In [243]:
a3=pd.read_excel('Unseen_Dataset.xlsx')

In [244]:
a3.head()

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,...,PL_Flag,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,MARITALSTATUS,EDUCATION,GENDER,last_prod_enq2,first_prod_enq2
0,0.000,0.0,0,0.000,0,0,0,4,1,4,...,1,0.000,0.0,1,0,Married,12TH,M,PL,PL
1,0.000,0.0,0,0.000,0,0,0,0,0,1,...,0,0.000,0.0,0,0,Single,GRADUATE,F,ConsumerLoan,ConsumerLoan
2,0.125,0.0,0,0.000,1,0,0,0,2,6,...,0,0.000,0.0,1,0,Married,SSC,M,ConsumerLoan,others
3,0.000,0.0,0,0.000,0,0,0,0,3,0,...,0,0.000,0.0,0,0,Married,POST-GRADUATE,M,AL,AL
4,0.000,0.0,1,0.167,0,0,0,0,6,0,...,0,0.429,0.0,1,0,Married,12TH,M,ConsumerLoan,PL


In [245]:
df.shape

(42064, 43)

In [246]:
df.columns[42]

'Approved_Flag'

In [247]:
cols_in_df = list(df.columns)
cols_in_df.pop(42)

'Approved_Flag'

In [249]:
df_unseen = a3[cols_in_df]

In [250]:
df_unseen['MARITALSTATUS'].unique()
df_unseen['EDUCATION'].unique()
df_unseen['GENDER'].unique()
df_unseen['last_prod_enq2'].unique()
df_unseen['first_prod_enq2'].unique()

array(['PL', 'ConsumerLoan', 'others', 'AL', 'HL', 'CC'], dtype=object)

In [251]:
df_unseen.loc[df_unseen['EDUCATION'] == 'SSC',['EDUCATION']]            = 1
df_unseen.loc[df_unseen['EDUCATION'] == '12TH',['EDUCATION']]           = 2
df_unseen.loc[df_unseen['EDUCATION'] == 'GRADUATE',['EDUCATION']]       = 3
df_unseen.loc[df_unseen['EDUCATION'] == 'UNDER GRADUATE',['EDUCATION']] = 3
df_unseen.loc[df_unseen['EDUCATION'] == 'POST-GRADUATE',['EDUCATION']]  = 4
df_unseen.loc[df_unseen['EDUCATION'] == 'OTHERS',['EDUCATION']]         = 1
df_unseen.loc[df_unseen['EDUCATION'] == 'PROFESSIONAL',['EDUCATION']]   = 3

In [252]:
df_unseen['EDUCATION'].value_counts()
df_unseen['EDUCATION'] = df_unseen['EDUCATION'].astype(int)
df_unseen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 42 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   pct_tl_open_L6M            100 non-null    float64
 1   pct_tl_closed_L6M          100 non-null    float64
 2   Tot_TL_closed_L12M         100 non-null    int64  
 3   pct_tl_closed_L12M         100 non-null    float64
 4   Tot_Missed_Pmnt            100 non-null    int64  
 5   CC_TL                      100 non-null    int64  
 6   Home_TL                    100 non-null    int64  
 7   PL_TL                      100 non-null    int64  
 8   Secured_TL                 100 non-null    int64  
 9   Unsecured_TL               100 non-null    int64  
 10  Other_TL                   100 non-null    int64  
 11  Age_Oldest_TL              100 non-null    int64  
 12  Age_Newest_TL              100 non-null    int64  
 13  time_since_recent_payment  100 non-null    int64  


In [253]:
df_encoded_unseen = pd.get_dummies(df_unseen, columns=['MARITALSTATUS','GENDER', 'last_prod_enq2' ,'first_prod_enq2'])


df_encoded_unseen.info()
k = df_encoded_unseen.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 54 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pct_tl_open_L6M               100 non-null    float64
 1   pct_tl_closed_L6M             100 non-null    float64
 2   Tot_TL_closed_L12M            100 non-null    int64  
 3   pct_tl_closed_L12M            100 non-null    float64
 4   Tot_Missed_Pmnt               100 non-null    int64  
 5   CC_TL                         100 non-null    int64  
 6   Home_TL                       100 non-null    int64  
 7   PL_TL                         100 non-null    int64  
 8   Secured_TL                    100 non-null    int64  
 9   Unsecured_TL                  100 non-null    int64  
 10  Other_TL                      100 non-null    int64  
 11  Age_Oldest_TL                 100 non-null    int64  
 12  Age_Newest_TL                 100 non-null    int64  
 13  time_s

In [254]:
model = xgb.XGBClassifier(objective='multi:softmax',
                                     num_class=4,
                                     colsample_bytree=0.9,
                                     learning_rate=1,
                                     max_depth=3,
                                     alpha=10,
                                     n_estimators=100)

In [255]:
model.fit(x_train, y_train)

XGBClassifier(alpha=10, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, num_class=4, ...)

In [261]:
y_pred_unseen = model.predict(df_encoded_unseen)

In [262]:
a3['Target_variable'] = y_pred_unseen

## Importing the Data into new Excel file named "Final_Prediction"

In [264]:
a3.to_excel ('final_prediction1.xlsx', index=False)

C:\Users\ranig\AppData\Local\Temp\ipykernel_19120\3435872899.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  a3.to_excel ('final_prediction1.xlsx', index=False)


In [259]:
## Program ends here, press Enter
end_time = time.time()
elapsed_time = end_time - start_time
print("Total run time of the program- " + str(round(elapsed_time, 2)) + ' sec')

input('Press Enter to exit')

Total run time of the program- 2150.44 sec
Press Enter to exit


''